In [1]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
import io
import pyarrow.parquet as pq
import pyarrow as pa
import re
import os

# Credenciais
account_name = 'uvabr'
account_key = os.environ['ACCOUNT_KEY']
container_name_read = 'silver'
container_name_send = 'gold'

endpoints = [
    "deputadosDespesas",
    "proposicoes",
    "proposicoesTemas",
    "proposicoesAutores",
    "frentes",
    "frentesDeputados",
    "grupos",
    "gruposMembros",
    "gruposHistorico",
    "legislaturas",
    "legislaturasMesas",
    "orgaos",
    "orgaosDeputados",
    "deputados",
    "deputadosOcupacoes",
    "deputadosProfissoes",
    "eventos",
    "eventosOrgaos",
    "eventosRequerimentos",
    "votacoes",
    "votacoesOrientacoes",
    "votacoesVotos",
    "votacoesObjetos",
    "votacoesProposicoes",
    "funcionarios",
    "licitacoes",
    "licitacoesContratos",
    "licitacoesItens",
    "licitacoesPedidos",
    "licitacoesPropostas",
    "tecadTermos",
    "tecadCategorias"
]

def ler_parquet_azure(account_name, account_key, container_name_read, file_name):
    # Conecte-se ao serviço de armazenamento de blob
    connect_str = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)

    # Acesse o contêiner
    container_client = blob_service_client.get_container_client(container_name_read)

    # Baixe o blob do Azure Blob Storage
    blob_client = container_client.get_blob_client(file_name)
    blob_data = blob_client.download_blob()
    blob_stream = io.BytesIO()
    blob_stream.write(blob_data.readall())

    # Leia o arquivo Parquet usando pyarrow
    parquet_table = pq.read_table(blob_stream)

    # Converta para DataFrame do Pandas
    df = parquet_table.to_pandas()

    return df

def enviar_parquet_azure(account_name, account_key, container_name_send, file_name, dataframe):
    # Conecte-se ao serviço de armazenamento de blob
    connect_str = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)

    # Acesse o contêiner
    container_client = blob_service_client.get_container_client(container_name_send)

    # Converta DataFrame para tabela Parquet
    table = pa.Table.from_pandas(dataframe)

    # Crie um buffer de memória para escrever os dados Parquet
    parquet_buffer = io.BytesIO()
    pq.write_table(table, parquet_buffer)

    # Envie os dados para o Azure Blob Storage
    blob_client = container_client.get_blob_client(file_name)
    blob_client.upload_blob(parquet_buffer.getvalue(), overwrite=True)

    print(f"Arquivo Parquet '{file_name}' enviado com sucesso para o Azure Blob Storage.")
        
def processar_endpoints(account_name, account_key, container_name_read, container_name_send, endpoints):
    for endpoint in endpoints:
        print(f"Leitura do {endpoint}.parquet")
        df = ler_parquet_azure(account_name, account_key, container_name_read, f'{endpoint}.parquet')
        enviar_parquet_azure(account_name, account_key, container_name_send, f'{endpoint}.parquet', df)
        print(f"envio do {endpoint}.parquet concluida")
        
processar_endpoints(account_name, account_key, container_name_read, container_name_send, endpoints)

KeyError: 'ACCOUNT_KEY'